In [1]:
# Parameters
end_period = "2016-09-30"
smoothing = True


## HLW

In [2]:
from bok_da.ts.nir import HLW

In [3]:
import pandas as pd
import numpy as np
import datetime

In [4]:
KORData = pd.read_excel("data/nir_data_all.xlsx", sheet_name="raw_data")
end_date = datetime.datetime.strptime('2024-07-01', '%Y-%m-%d')
KORData_before = KORData[KORData['Date'] <= end_date]
data = KORData_before.set_index("Date")
data_list = ['rgdp', 'inf1', 'interest', 'infe']
data = data[data_list]
data['inf1'] = data['inf1'].pct_change(4) * 100

if smoothing:
    data['inf1'] = data['inf1'].rolling(4).mean() # 4분기 이동평균
    
data['rgdp'] = np.log(data['rgdp']) * 400
data['interest'] = data['interest'] - data['inf1']
data = data.drop('infe', axis=1)
data = data.dropna()
#data = data.loc['1991-01-01':]
KORmacro = data['2000-01-01':].copy()
#KORmacro = data.copy()
def shift_quarter_end(date):
    if date.month == 1:
        return date.replace(month=3, day=31)
    elif date.month == 4:
        return date.replace(month=6, day=30)
    elif date.month == 7:
        return date.replace(month=9, day=30)
    else:
        return date.replace(month=12, day=31)

KORmacro.index = KORmacro.index.map(shift_quarter_end)
KORmacro

,rgdp,inf1,interest
Date,,,
2000-03-31,4961.268953,0.422939,4.527061
2000-06-30,4967.384872,0.888428,4.211572
2000-09-30,4978.220343,1.394206,3.725794
2000-12-31,4976.889295,1.869223,3.460777
2001-03-31,4980.892991,2.376812,2.763188
...,...,...,...
2023-09-30,5296.439326,4.367675,-0.819675
2023-12-31,5298.255260,4.007923,-0.431923
2024-03-31,5303.422191,3.466374,0.039626


In [5]:
mcmc =  {
            'sig2_g': 0,
            'sig2_yt': 0,
            'sig2_yc': 0,
            'sig2_z': 0,
            'sig2_p': 0,
            'slope_yc': 0,
            'slope_z': 0,
            'slope_p': 0
        }

prior = {
            'yc_mean': np.array([0.7, 0.2, -0.3]), # 0.7, 0, 0// 0.7, 0.2, -0.4, -0.2. -0.15, -0.25
            'yc_variance': np.diag(np.array([0.1, 0.1, 0.03]) ** 2), # 0.1, 0.1, 0.03
            'sig2_yc_shape': 5,
            'sig2_yc_scale': 4 * 2**2,
            'sig2_g_shape': 5,
            'sig2_g_scale': 4 * 0.1**2,
            'sig2_yt_shape': 5,
            'sig2_yt_scale': 4 * 2**2,
            'sig2_z_shape': 5,
            'sig2_z_scale': 4 * 0.5**2,
            'p_mean': np.array([0.7, 0.1]), # 0.5, 0
            'p_variance': np.diag(np.array([0.1, 0.1]) ** 2), # 1, 1
            'sig2_p_shape': 5,
            'sig2_p_scale': 4 * 0.1**2
}
        
model = HLW(n_draws=5000, burnin=1000, MCMCAlgorithm=mcmc, PriorPara=prior, verbose=True, ZSpec='rw')

> Initializing Model...
  - Total MCMC iterations: 5000
  - Burn-in period: 1000
  - Latent variables percentiles: [5, 95]
  - Parameters percentiles: [0.05, 0.95]
  - Interest rate z-component specification: rw


In [6]:
#end_date = '2015-12-31'
print(end_period)
model.fit(KORmacro.loc[:end_period], dates=KORmacro.loc[:end_period].index)


2016-09-30
> Starting model fitting...
  - Data shape: (67, 3)
  - Dates provided with length: 67


Sampling:   0%|          | 0/5000 [00:00<?, ?it/s]

Step 1/3 :   0%|          | 0/5000 [00:00<?, ?it/s]

Step 1/3 :   0%|          | 0/5000 [00:00<?, ?it/s]

Step 1/3 :   0%|          | 2/5000 [00:00<04:32, 18.35it/s]

...

Step 3/3 :  87%|████████▋ | 4369/5000 [01:00<00:07, 88.16it/s]

Step 3/3 :  87%|████████▋ | 4369/5000 [01:00<00:07, 88.16it/s]

Step 3/3 :  88%|████████▊ | 4378/5000 [01:00<00:07, 84.86it/s]

Step 3/3 :  88%|████████▊ | 4378/5000 [01:00<00:07, 84.86it/s]

Step 3/3 :  88%|████████▊ | 4378/5000 [01:00<00:07, 84.86it/s]

In [ ]:
rstar_HLW = pd.DataFrame(model.SummaryLV['Avg']['rNatural'])
rstar_HLW.index = KORmacro.loc[KORmacro.index[4]:end_period].index
if smoothing:
    rstar_HLW.columns = ['rstar_HLW_inf_smoothed']
else:
    rstar_HLW.columns = ['rstar_HLW']
rstar = rstar_HLW.reset_index().rename(columns={'index': 'Date'}).to_json(orient='records', date_format='iso')
rstar_HLW.reset_index().rename(columns={'index': 'Date'}) \
    .assign(Date=lambda df: df['Date'].dt.strftime('%Y-%m-%d')) \
    .to_json('rstar_HLW.json', orient='records')